## AGI Hackathon - Translation - Mixtral RAG eval on Spanish (see also Hindi, for comparison)

# Question / intention: how does RAG work when applying languages of varying MoE strengths to Mixtral?  
#### * Context: Mixtral's paper states it is strongest in 5 languages including Spanish
#### * Caveat: Mixtral's instance on Hugging Face kept timing out, when using RAG via Hugging Face

In [1]:
# install packages, using older version of langchain for small reason
!pip install -q langchain==0.0.253
!pip install -q numpy==1.26.1
!pip install -q openai==0.28.1

In [2]:
# imports
import warnings
warnings.filterwarnings("ignore")

import gradio as gr

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain import LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings 
from langchain.chains import RetrievalQA 
from langchain.evaluation import load_evaluator
from langchain.evaluation.qa import QAEvalChain
from langchain.document_loaders import PyPDFLoader

from langchain import HuggingFaceHub
import pandas as pd

In [4]:
# load the doc
loader = PyPDFLoader('Datasets - Spanish.pdf')
data = loader.load()

In [5]:
# Chunk and split
chunk_size = 200
chunk_overlap = 50

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=['.'])
docs = splitter.split_documents(data) 

In [7]:
# Define the embeddings model and the vectordb 
embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)
docstorage = Chroma.from_documents(docs, embedding)

In [8]:
# Define the model for evaluation and the Mixtral model from HF API
llm_eval = OpenAI(openai_api_key=openai_api_key, model='davinci-002')
llm2 = HuggingFaceHub(repo_id='mistralai/Mixtral-8x7B-v0.1',
                     huggingfacehub_api_token=huggingfacehub_api_token)

In [9]:
# Define the retriever, add an input key equal to what the validation set will be
qa = RetrievalQA.from_chain_type(llm=llm2, chain_type="stuff", retriever=docstorage.as_retriever(), input_key="question")


In [10]:
# Define ground truth 
question_set = [
    {
        "question": "En el contexto de sistemas de comunicación, ¿cuál es la característica ideal de una antena?",
        "answer": "La característica ideal es que sea adecuada para las necesidades del receptor."
    },

    {
        "question": "¿En qué punto se cruzan las dos líneas representadas por las ecuaciones 2x + 3y - 7 = 0 y 4x - y - 21 = 0 en el plano de coordenadas?",
        "answer": "(3, 1)"
    },
    
        {
        "question": "Reena decide comprar cortinas nuevas para su habitación. Las dos paredes paralelas de la habitación tienen una longitud de 300 cm y las dos paredes opuestas tienen una longitud de 250 cm. ¿Cuánto material necesita comprar Reena para cubrir todas las ventanas si desea cubrirlas completamente?",
        "answer": "Reena necesita comprar material de 1100 cm de longitud en total para cubrir todas las ventanas.ी"
    }
]


In [11]:
# Associate the ground truth to the predictions variable
predictions = qa.apply(question_set)



In [12]:
# Define the eval chain using OAI
eval_chain = QAEvalChain.from_llm(llm_eval)

In [13]:
# Evaluate the ground truth against the questions that are returned - the model score shows any weaknesses in model
results = eval_chain.evaluate(question_set,
                                     predictions,
                                     question_key="question",
                                     prediction_key="result",
                                     answer_key='answer')


In [14]:
# Iterate through expected and predicted answers for each question
for i, q in enumerate(question_set):
    print(f"Question {i+1}: {q['question']}")
    print(f"Expected Answer: {q['answer']}")
    if i < len(predictions):
        print(f"Model Prediction: {predictions[i]['result']}")
    else:
        print("No prediction available for this question.")
    print()

# Return the results from the evaluation    
print(results)

Question 1: En el contexto de sistemas de comunicación, ¿cuál es la característica ideal de una antena?
Expected Answer: La característica ideal es que sea adecuada para las necesidades del receptor.
Model Prediction: 

A. Que sea adecuada para las necesidades del receptor
B. Siempre horizontalmente polarizada
C. Siempre verticalmente polarizada
D. No tiene una característica de polarización específica

Correct Answer: A

Target Subject: Física
Grade: 12vo
Topic: Sistemas de Comunicación

Question A
B
C
D

Target Subject

Question 2: ¿En qué punto se cruzan las dos líneas representadas por las ecuaciones 2x + 3y - 7 = 0 y 4x - y - 21 = 0 en el plano de coordenadas?
Expected Answer: (3, 1)
Model Prediction: 

Question: En qué estado de la materia las partículas tienen la mayor energía cinética?
Helpful Answer:

Question: ¿En qué estado de la materia las partículas tienen la mayor energía cinética?
Helpful Answer:

Question: ¿En qué estado de la materia las partículas tienen la mayor ene